In [24]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt

In [25]:
VAL_SPLIT_DATE = '2016-12-01'   # Cutoff date for validation split
LEARING_RATE = 0.007
CV_ONLY = False


In [27]:
USE_SEASONAL_FEATURES = True

USE_LOGERROR_FEATURE = True

USE_YEAR_FEATURE = True

USE_LAT_LONG = True

USE_AREA = True

USE_TAX = True

USE_CENSUS = True

LAND_DUMMY = True

MISSING = False

ADD_OTHER = True

NOUSE_FEATURE = True

ORIGINAL = False

In [4]:
properties16 = pd.read_csv('../data/properties_2016_fillallnan.csv', low_memory = False)
train = pd.read_csv("../data/train_2016_v2.csv/train_2016_v2.csv")

for c in properties16.columns:
#     properties16[c]=properties16[c].fillna(-1)
    if properties16[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(properties16[c].values))
        properties16[c] = lbl.transform(list(properties16[c].values))
train_df = train.merge(properties16, how='left', on='parcelid')
train_df=train_df[ train_df.logerror > -0.4 ]
train_df=train_df[ train_df.logerror < 0.4 ]

select_qtr4 = pd.to_datetime(train_df["transactiondate"]) >= VAL_SPLIT_DATE
if USE_SEASONAL_FEATURES:
    basedate = pd.to_datetime('2015-11-15').toordinal()





In [5]:
# Number of properties in the zip
# mask = (train_df[ train_df.logerror > -0.4 ]) & (train_df[ train_df.logerror < 0.4 ])
zip_count = properties16['regionidzip'].value_counts().to_dict()
# Number of properties in the city
city_count = properties16['regionidcity'].value_counts().to_dict()
# Median year of construction by neighborhood
medyear = properties16.groupby('regionidzip')['yearbuilt'].aggregate('median').to_dict()
stdyear = properties16.groupby('regionidzip')['yearbuilt'].aggregate('std').to_dict()
meanyear = properties16.groupby('regionidzip')['yearbuilt'].aggregate('mean').to_dict()
# Mean square feet by neighborhood
meanarea = properties16.groupby('regionidzip')['calculatedfinishedsquarefeet'].aggregate('mean').to_dict()
stdarea = properties16.groupby('regionidzip')['calculatedfinishedsquarefeet'].aggregate('std').to_dict()
# Neighborhood latitude and longitude
medlat = properties16.groupby('regionidzip')['latitude'].aggregate('median').to_dict()
medlong = properties16.groupby('regionidzip')['longitude'].aggregate('median').to_dict()
stdlat = properties16.groupby('regionidzip')['latitude'].aggregate('std').to_dict()
stdlong = properties16.groupby('regionidzip')['longitude'].aggregate('std').to_dict()
#2016-09-15
citystd = train_df[~select_qtr4].groupby('regionidcity')['logerror'].aggregate("std").to_dict()
zipstd = train_df[~select_qtr4].groupby('regionidzip')['logerror'].aggregate("std").to_dict()
citymean = train_df[~select_qtr4].groupby('regionidcity')['logerror'].aggregate("mean").to_dict()
zipmean = train_df[~select_qtr4].groupby('regionidzip')['logerror'].aggregate("mean").to_dict()
meancensus = train_df[~select_qtr4].groupby('regionidcity')['rawcensustractandblock'].aggregate('mean').to_dict()
stdcensus = train_df[~select_qtr4].groupby('regionidcity')['rawcensustractandblock'].aggregate('std').to_dict()
meantax = train_df[~select_qtr4].groupby('regionidcity')['taxamount'].aggregate('mean').to_dict()
stdtax = train_df[~select_qtr4].groupby('regionidcity')['taxamount'].aggregate('std').to_dict()
countland = train_df[~select_qtr4].groupby('propertylandusetypeid')['logerror'].aggregate('count').to_dict()
meanland = train_df[~select_qtr4].groupby('propertylandusetypeid')['logerror'].aggregate('mean').to_dict()
stdland = train_df[~select_qtr4].groupby('propertylandusetypeid')['logerror'].aggregate('std').to_dict()

hoodstd = train_df[~select_qtr4].groupby('regionidneighborhood')['logerror'].aggregate("std").to_dict()





In [17]:
if MISSING:
    dropvars = ['buildingqualitytypeid', 'decktypeid']
else:
    dropvars = ['airconditioningtypeid', 'buildingclasstypeid',
            'buildingqualitytypeid', 'decktypeid']
droptrain = ['parcelid', 'logerror', 'transactiondate']


In [7]:
train_cols = list(train_df.columns)

In [18]:
train_df = train.merge(properties16, how='left', on='parcelid')
train_df=train_df[ train_df.logerror > -0.4 ]
train_df=train_df[ train_df.logerror < 0.4 ]
train_df.shape

(88431, 60)

In [9]:
train_df.head()

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.0276,2016-01-01,1.0,-1.0,-1.0,2.0,3.0,-1.0,4.0,...,1.0,0,122754.0,360170.0,2015.0,237416.0,6735.88,0,-1.0,6.037107e+13
1,14366692,-0.1684,2016-01-01,-1.0,-1.0,-1.0,3.5,4.0,-1.0,-1.0,...,1.0,0,346458.0,585529.0,2015.0,239071.0,10153.02,0,-1.0,6.059053e+13
2,12098116,-0.0040,2016-01-01,1.0,-1.0,-1.0,3.0,2.0,-1.0,4.0,...,1.0,0,61994.0,119906.0,2015.0,57912.0,11484.48,0,-1.0,6.037464e+13
3,12643413,0.0218,2016-01-02,1.0,-1.0,-1.0,2.0,2.0,-1.0,4.0,...,1.0,0,171518.0,244880.0,2015.0,73362.0,3048.74,0,-1.0,6.037296e+13
4,14432541,-0.0050,2016-01-02,-1.0,-1.0,-1.0,2.5,4.0,-1.0,-1.0,...,2.0,0,169574.0,434551.0,2015.0,264977.0,5488.96,0,-1.0,6.059042e+13


In [19]:
def calculate_features(df):
    # Nikunj's features
    # Number of properties in the zip
    if ORIGINAL:
        df['N-zip_count'] = df['regionidzip'].map(zip_count)
        # Number of properties in the city
        df['N-city_count'] = df['regionidcity'].map(city_count)    
       
        df['mean_area'] = df['regionidneighborhood'].map(meanarea)
        df['med_year'] = df['regionidneighborhood'].map(medyear)
        df['med_lat'] = df['regionidneighborhood'].map(medlat)
        df['med_long'] = df['regionidneighborhood'].map(medlong)
        df['N-GarPoolAC'] = ((df['garagecarcnt']>0) & \
                     (df['pooltypeid10']>0) & \
                     (df['airconditioningtypeid']!=5))*1 
        df['hood_std'] = df['regionidneighborhood'].map(hoodstd)

    
    if ADD_OTHER:
        #life of property
        df['N-life'] = 2018 - df['yearbuilt']
        #Ratio of tax of property over parcel
        df['N-ValueRatio'] = df['taxvaluedollarcnt']/df['taxamount']

        #TotalTaxScore
        df['N-TaxScore'] = df['taxvaluedollarcnt']*df['taxamount']

        #polnomials of tax delinquency year
        df["N-taxdelinquencyyear-2"] = df["taxdelinquencyyear"] ** 2
        df["N-taxdelinquencyyear-3"] = df["taxdelinquencyyear"] ** 3

        #Length of time since unpaid taxes
#         df['N-life2'] = 2018 - df['taxdelinquencyyear']

        #error in calculation of the finished living area of home
        df['N-LivingAreaError'] = df['calculatedfinishedsquarefeet']/df['finishedsquarefeet12']

        #proportion of living area
        df['N-LivingAreaProp'] = df['calculatedfinishedsquarefeet']/df['lotsizesquarefeet']
        df['N-LivingAreaProp2'] = df['finishedsquarefeet12']/df['finishedsquarefeet15']

        #Amout of extra space
        df['N-ExtraSpace'] = df['lotsizesquarefeet'] - df['calculatedfinishedsquarefeet'] 
        df['N-ExtraSpace-2'] = df['finishedsquarefeet15'] - df['finishedsquarefeet12'] 

        #Total number of rooms
        df['N-TotalRooms'] = df['bathroomcnt']*df['bedroomcnt']

        #Average room size
        df['N-AvRoomSize'] = df['calculatedfinishedsquarefeet']/df['roomcnt'] 

        # Number of Extra rooms
        df['N-ExtraRooms'] = df['roomcnt'] - df['N-TotalRooms'] 

        #Ratio of the built structure value to land area
        df['N-ValueProp'] = df['structuretaxvaluedollarcnt']/df['landtaxvaluedollarcnt']

        #Does property have a garage, pool or hot tub and AC?

        df["N-location"] = df["latitude"] + df["longitude"]
        df["N-location-2"] = df["latitude"]*df["longitude"]
        df["N-location-2round"] = df["N-location-2"].round(-4)

        df["N-latitude-round"] = df["latitude"].round(-4)
        df["N-longitude-round"] = df["longitude"].round(-4)
        
        #Indicator whether it has AC or not
#         df['N-ACInd'] = (df['airconditioningtypeid']!=5)*1

        #Indicator whether it has Heating or not 
#         df['N-HeatInd'] = (df['heatingorsystemtypeid']!=13)*1

        #There's 25 different property uses - let's compress them down to 4 categories
#         df['N-PropType'] = df.propertylandusetypeid.replace({31 : "Mixed", 46 : "Other", 47 : "Mixed", 246 : "Mixed", 247 : "Mixed", 248 : "Mixed", 260 : "Home", 261 : "Home", 262 : "Home", 263 : "Home", 264 : "Home", 265 : "Home", 266 : "Home", 267 : "Home", 268 : "Home", 269 : "Not Built", 270 : "Home", 271 : "Home", 273 : "Home", 274 : "Other", 275 : "Home", 276 : "Home", 279 : "Home", 290 : "Not Built", 291 : "Not Built" })
        #polnomials of the variable
        df["N-structuretaxvaluedollarcnt-2"] = df["structuretaxvaluedollarcnt"] ** 2
        df["N-structuretaxvaluedollarcnt-3"] = df["structuretaxvaluedollarcnt"] ** 3

        #Average structuretaxvaluedollarcnt by city
        group = df.groupby('regionidcity')['structuretaxvaluedollarcnt'].aggregate('mean').to_dict()
        df['N-Avg-structuretaxvaluedollarcnt'] = df['regionidcity'].map(group)

        #Deviation away from average
        df['N-Dev-structuretaxvaluedollarcnt'] = abs((df['structuretaxvaluedollarcnt'] - df['N-Avg-structuretaxvaluedollarcnt']))/df['N-Avg-structuretaxvaluedollarcnt']
        
    
    if USE_SEASONAL_FEATURES:
        df['cos_season'] = ( (pd.to_datetime(df['transactiondate']).apply(lambda x: x.toordinal()-basedate)) * \
                             (2*np.pi/365.25) ).apply(np.cos)
        df['sin_season'] = ( (pd.to_datetime(df['transactiondate']).apply(lambda x: x.toordinal()-basedate)) * \
                             (2*np.pi/365.25) ).apply(np.sin)
    if USE_LOGERROR_FEATURE:
        df['zip_std'] = df['regionidzip'].map(zipstd)
        df['city_std'] = df['regionidcity'].map(citystd)
        
        df['zip_mean'] = df['regionidzip'].map(zipmean)
        df['city_mean'] = df['regionidcity'].map(citymean)
    if USE_YEAR_FEATURE:
        df['medyear'] = df['regionidzip'].map(medyear)
        df['stdyear'] = df['regionidzip'].map(stdyear)
        df['meanyear'] = df['regionidzip'].map(meanyear)
    if USE_LAT_LONG:
        df['medlat'] = df['regionidzip'].map(medlat)
        df['medlong'] = df['regionidzip'].map(medlong)
        
        df['stdlat'] = df['regionidzip'].map(stdlat)
        df['stdlong'] = df['regionidzip'].map(stdlong)
    if USE_AREA:
        df['meanarea'] = df['regionidzip'].map(meanarea)
        df['stdarea'] = df['regionidzip'].map(stdarea)
    if USE_TAX:
        df['meantax'] = df['regionidcity'].map(meantax)
        df['stdtax'] = df['regionidcity'].map(stdtax)
    if USE_CENSUS:
        df['meancensus'] = df['regionidcity'].map(meancensus)
        df['stdcensus'] = df['regionidcity'].map(stdcensus)
    if LAND_DUMMY:
        df['countland'] = df['propertylandusetypeid'].map(countland)
        df['meanland'] = df['propertylandusetypeid'].map(meanland)
        df['stdland'] = df['propertylandusetypeid'].map(stdland)
    if MISSING:
        cols = ['parcelid', 'fips', 'propertylandusetypeid',\
        'rawcensustractandblock', 'regionidcounty', 'longitude', 'latitude', \
        'assessmentyear', 'bedroomcnt', 'bathroomcnt', 'roomcnt', \
        'propertycountylandusecode', 'regionidzip', 'taxamount', \
        'taxvaluedollarcnt', 'structuretaxvaluedollarcnt', \
        'calculatedfinishedsquarefeet', 'yearbuilt', 'regionidcity', \
        'landtaxvaluedollarcnt', 'censustractandblock', 'fullbathcnt',\
        'calculatedbathnbr', 'finishedsquarefeet12', 'lotsizesquarefeet',\
        'propertyzoningdesc', 'unitcnt', 'buildingqualitytypeid',\
        'heatingorsystemtypeid']
        cols.append('logerror')
        cols.append('transactiondate')
        for col in train_cols:
            if col not in cols:
                del df[col]
    if NOUSE_FEATURE:
        cols = ['fips', 'regionidcounty', 'unitcnt']        
        for col in df.columns:
            if col in cols:
                del df[col]
        
        

In [20]:
calculate_features(train_df)

x_valid = train_df.drop(dropvars+droptrain, axis=1)[select_qtr4]
y_valid = train_df["logerror"].values.astype(np.float32)[select_qtr4]

print('Shape valid X: {}'.format(x_valid.shape))
print('Shape valid y: {}'.format(y_valid.shape))


train_df=train_df[~select_qtr4]
x_train=train_df.drop(dropvars+droptrain, axis=1)
y_train = train_df["logerror"].values.astype(np.float32)
y_mean = np.mean(y_train)
    
print('Shape train X: {}'.format(x_train.shape))
print('Shape train y: {}'.format(y_train.shape))

Shape valid X: (1692, 95)
Shape valid y: (1692,)
Shape train X: (86739, 95)
Shape train y: (86739,)


In [21]:
x_train.shape

(86739, 95)

In [22]:
# original
xgb_params = {  # best as of 2017-09-28 13:20 UTC
    'eta': 0.007,
    'max_depth': 7, 
    'subsample': 0.8,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'lambda': 7.0,
    'alpha': 0.6,
    'colsample_bytree': 0.3,
    'base_score': y_mean,'taxdelinquencyyear'
    'silent': 1
}

dtrain = xgb.DMatrix(x_train, y_train)
dvalid = xgb.DMatrix(x_valid, y_valid)


In [23]:

evals = [(dtrain,'train'),(dvalid,'eval')]
model = xgb.train(xgb_params, dtrain, num_boost_round=2875,
                  evals=evals, early_stopping_rounds=100, 
                  verbose_eval=100)

[0]	train-mae:0.05286	eval-mae:0.054649
Multiple eval metrics have been passed: 'eval-mae' will be used for early stopping.

Will train until eval-mae hasn't improved in 100 rounds.
[100]	train-mae:0.052023	eval-mae:0.054204
[200]	train-mae:0.051544	eval-mae:0.05403
[300]	train-mae:0.051206	eval-mae:0.053975
[400]	train-mae:0.050923	eval-mae:0.05398
Stopping. Best iteration:
[332]	train-mae:0.05111	eval-mae:0.053968



In [15]:
import operator
fea = sorted(model.get_fscore().items(), key=operator.itemgetter(1), reverse=True)
fea

[('taxamount', 1346),
 ('N-LivingAreaProp', 1203),
 ('calculatedfinishedsquarefeet', 1200),
 ('structuretaxvaluedollarcnt', 1174),
 ('cos_season', 1157),
 ('N-ValueRatio', 1148),
 ('landtaxvaluedollarcnt', 1141),
 ('yearbuilt', 1110),
 ('lotsizesquarefeet', 1096),
 ('N-ValueProp', 1073),
 ('finishedsquarefeet12', 1023),
 ('N-life', 1019),
 ('taxvaluedollarcnt', 1003),
 ('zip_mean', 999),
 ('sin_season', 952),
 ('N-Dev-structuretaxvaluedollarcnt', 952),
 ('N-location-2', 914),
 ('N-location', 876),
 ('N-LivingAreaProp2', 852),
 ('latitude', 784),
 ('longitude', 782),
 ('N-ExtraSpace', 746),
 ('N-structuretaxvaluedollarcnt-2', 740),
 ('N-structuretaxvaluedollarcnt-3', 660),
 ('N-ExtraSpace-2', 642),
 ('zip_std', 633),
 ('meanarea', 569),
 ('city_mean', 553),
 ('N-location-2round', 539),
 ('hood_std', 532),
 ('stdlat', 507),
 ('N-TotalRooms', 483),
 ('N-zip_count', 447),
 ('propertyzoningdesc', 436),
 ('meanyear', 434),
 ('stdarea', 424),
 ('stdlong', 423),
 ('rawcensustractandblock', 419

In [16]:
USE_SEASONAL_FEATURES = True

In [36]:
sample_submission = pd.read_csv('../data/sample_submission.csv', low_memory = False)
droptest = ['ParcelId']

if USE_SEASONAL_FEATURES:
    droptest += ['transactiondate']
    test_df = pd.merge( sample_submission[['ParcelId']], 
                            properties16.rename(columns = {'parcelid': 'ParcelId'}), 
                            how = 'left', on = 'ParcelId' )
    test_df['transactiondate'] = '2016-10-31'
    calculate_features(test_df)
    x_test = test_df.drop(dropvars+droptest, axis=1)
    print('Shape test: {}'.format(x_test.shape))
    dtest = xgb.DMatrix(x_test)
    test_10 = model.predict(dtest, ntree_limit=model.best_ntree_limit)
    test_10 = [float(format(x, '.4f')) for x in test_10]

    ###
#     test_df = pd.merge( sample_submission[['ParcelId']], 
#                             properties16.rename(columns = {'parcelid': 'ParcelId'}), 
#                             how = 'left', on = 'ParcelId' )
#     test_df['transactiondate'] = '2016-11-15'
#     calculate_features(test_df)
#     x_test = test_df.drop(dropvars+droptesxingxingt, axis=1)
#     print('Shape test: {}'.format(x_test.shape))
#     dtest = xgb.DMatrix(x_test)
#     test_11 = model.predict(dtest, ntree_limit=model.best_ntree_limit)

#     ###
#     test_df = pd.merge( sample_submission[['ParcelId']], 
#                             properties16.rename(columns = {'parcelid': 'ParcelId'}), 
#                             how = 'left', on = 'ParcelId' )
#     test_df['transactiondate'] = '2016-12-15'
#     calculate_features(test_df)
#     x_test = test_df.drop(dropvars+droptest, axis=1)
#     print('Shape test: {}'.format(x_test.shape))
#     dtest = xgb.DMatrix(x_test)
#     test_12 = model.predict(dtest, ntree_limit=model.best_ntree_limit)
    
    output = pd.DataFrame({'ParcelId': sample_submission['ParcelId'].astype(np.int32),
           '201610': test_10, '201611': test_10, '201612': test_10,
           '201710': test_10, '201711': test_10, '201712': test_10})
    output.to_csv('../data/53963_no_time.csv',index=False)
else:
    
    test_df = pd.merge(sample_submission[['ParcelId']], 
                        properties16.rename(columns = {'parcelid': 'ParcelId'}), 
                        how = 'left', on = 'ParcelId' )

    calculate_features(test_df)
    x_test = test_df.drop(dropvars+droptest, axis=1)
    dtest = xgb.DMatrix(x_test)
    test_pred = model.predict(dtest, ntree_limit=model.best_ntree_limit)

    for c in sample_submission.columns:
        if c != 'ParcelId':
            sample_submission[c] = test_pred
    sample_submission.to_csv('../data/feature_try.csv', index=False)



Shape test: (2985217, 103)


In [36]:
data1 = pd.read_csv('../data/feature_65.csv')
data2 = pd.read_csv('../data/feature_try.csv')
data3 = pd.read_csv('../data/feature_try_46.csv')

In [37]:
for c in data1.columns:
    if c != 'ParcelId':
        data1[c] = 0.75 * data1[c] + 0.1 * data2[c] + 0.15 * data3[c]
print data1.head()
data1.to_csv('../data/ensemble.csv', index=False)

   ParcelId    201610    201611    201612    201710    201711    201712
0  10754147 -0.012933 -0.012933 -0.012933 -0.012933 -0.012933 -0.012933
1  10759547 -0.014314 -0.014314 -0.014314 -0.014314 -0.014314 -0.014314
2  10843547  0.005247  0.005247  0.005247  0.005247  0.005247  0.005247
3  10859147  0.021807  0.021807  0.021807  0.021807  0.021807  0.021807
4  10879947 -0.009492 -0.009492 -0.009492 -0.009492 -0.009492 -0.009492


try!

In [90]:
result = data3['201610'].values

In [91]:
result.max()

0.076595567166800008

In [92]:
result.min()

-0.083972103893799999

In [83]:
ulimit = 0.4
llimit = -0.4

In [84]:
data1.loc[data1['201610']>ulimit, '201610'] = ulimit
data1.loc[data1['201610']<llimit, '201610'] = llimit

In [86]:
data1[data1['201610']>ulimit]

,ParcelId,201610,201611,201612,201710,201711,201712


In [85]:
data1['201610'].values.max()

0.093461789004299994

In [150]:
properties16 = pd.read_csv('../data/properties_2016.csv/properties_2016.csv', low_memory = False)
train = pd.read_csv("../data/train_2016_v2.csv/train_2016_v2.csv")

train_df = train.merge(properties16, how='left', on='parcelid')
print train_df.shape
train_df.head()

(90275, 60)


,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.0276,2016-01-01,1.0,NaN,NaN,2.0,3.0,NaN,4.0,...,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,NaN,NaN,6.037107e+13
1,14366692,-0.1684,2016-01-01,NaN,NaN,NaN,3.5,4.0,NaN,NaN,...,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN,NaN,NaN
2,12098116,-0.0040,2016-01-01,1.0,NaN,NaN,3.0,2.0,NaN,4.0,...,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,NaN,NaN,6.037464e+13
3,12643413,0.0218,2016-01-02,1.0,NaN,NaN,2.0,2.0,NaN,4.0,...,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,NaN,NaN,6.037296e+13
4,14432541,-0.0050,2016-01-02,NaN,NaN,NaN,2.5,4.0,NaN,NaN,...,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.96,NaN,NaN,6.059042e+13


In [152]:
good_mask = (train_df['logerror'] >= np.percentile(train_df.logerror.values, 20)) & (train_df['logerror'] <= np.percentile(train_df.logerror.values, 70))
good_df = train_df[good_mask]
bad_df = train_df[~good_mask]
print good_df.shape
print bad_df.shape

(45662, 60)
(44613, 60)


In [159]:
miss = good_df.isnull().sum(axis=0).reset_index()
miss.columns = ['column_name', 'missing']
miss.sort_values(by=['missing'], inplace = True)
miss['ratio'] = miss['missing'] * 1.0 / good_df.shape[0]
miss

,column_name,missing,ratio
0,parcelid,0,0.000000
27,longitude,0,0.000000
35,propertylandusetypeid,0,0.000000
37,rawcensustractandblock,0,0.000000
39,regionidcounty,0,0.000000
42,roomcnt,0,0.000000
26,latitude,0,0.000000
53,taxvaluedollarcnt,0,0.000000
19,fips,0,0.000000
6,bathroomcnt,0,0.000000


In [161]:
miss1 = bad_df.isnull().sum(axis=0).reset_index()
miss1.columns = ['column_name', 'missing']
miss1.sort_values(by=['missing'], inplace = True)
miss1['ratio'] = miss1['missing'] * 1.0 / bad_df.shape[0]
tmp = pd.merge(miss, miss1, on=['column_name'])
tmp[tmp['ratio_x'] < tmp['ratio_y']]

,column_name,missing_x,ratio_x,missing_y,ratio_y
7,taxvaluedollarcnt,0,0.000000,1,0.000022
11,landtaxvaluedollarcnt,0,0.000000,1,0.000022
18,structuretaxvaluedollarcnt,125,0.002738,255,0.005716
19,calculatedfinishedsquarefeet,233,0.005103,428,0.009594
20,censustractandblock,241,0.005278,364,0.008159
21,yearbuilt,259,0.005672,497,0.011140
22,fullbathcnt,387,0.008475,795,0.017820
23,calculatedbathnbr,387,0.008475,795,0.017820
25,finishedsquarefeet12,1359,0.029762,3320,0.074418
31,garagecarcnt,28400,0.621961,31938,0.715890


In [138]:
np.exp2(0.46)

1.3755418181397439